In [2]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


# Iterators, generators

In [1]:
for i in range(5): print(i, end=" ")
print()
for i in (0, 1, 2, 3, 4): print(i, end=" ")
print()
for i in {0, 1, 2, 3, 4}: print(i, end=" ")
list(map(type, (range(5), (0, 1, 2, 3, 4), {0, 1, 2, 3, 4})))

0 1 2 3 4 
0 1 2 3 4 
0 1 2 3 4 

[range, tuple, set]

## Iterable

In [2]:
hasattr(range(5), "__iter__"), hasattr(tuple(), "__iter__"), hasattr(set(), "__iter__")

(True, True, True)

```python
iterable.__iter__() -> iterator
```

## Iterator protocol

- `__iter__` - return the iterator object itself
- `__next__` - return the next item from the iterator

Return the next item from the container. If there are no further items, raise the `StopIteration` exception. Once an iterator’s `__next__` method raises `StopIteration`, it must continue to do so on subsequent calls.

In [3]:
for i in range(5):
    print(i, end=" ")
print()

iterator = range(5).__iter__()
while True:
    try:
        i = iterator.__next__()
        print(i, end=" ")
    except StopIteration:
        break

0 1 2 3 4 
0 1 2 3 4 

In [4]:
type(range(5)), type(iterator)

(range, range_iterator)

In [5]:
from collections import namedtuple
from typing import Iterator


Page = namedtuple("Page", ["text", "number"])


class Book:
    def __init__(self) -> None:
        self.pages = []

    def add_page(self, text: str) -> None:
        self.pages.append(
            Page(text, number=len(self.pages) + 1)
        )
        
    def __iter__(self) -> Iterator[Page]:
        return BookIter(self)
    

class BookIter:
    def __init__(self, book: Book) -> None:
        self.pages = book.pages
        self.book = book
        self._cursor = 0
        
    def __iter__(self) -> "BookIter":
        return self
    
    def __next__(self) -> Page:
        if len(self.pages) > self._cursor:     
            result = self.pages[self._cursor]  
            self._cursor += 1                  
            return result
        raise StopIteration                    

In [6]:
book = Book()
for i in range(1, 5): 
    book.add_page(f"page_{i}")

for page in book:
    print(page)

Page(text='page_1', number=1)
Page(text='page_2', number=2)
Page(text='page_3', number=3)
Page(text='page_4', number=4)


In [7]:
bookiter = iter(book)
print(next(bookiter))
print(next(bookiter))
print(next(bookiter))
print(next(bookiter))
print(next(bookiter))

Page(text='page_1', number=1)
Page(text='page_2', number=2)
Page(text='page_3', number=3)
Page(text='page_4', number=4)


StopIteration: 

## Why do we need our own BookIter?

In [8]:
class LazyBook(Book):
    def __iter__(self) -> Iterator[Page]:
        return iter(self.pages)

In [9]:
lazy_book = LazyBook()
for i in range(1, 5): 
    lazy_book.add_page(f"page_{i}")

for page in lazy_book:
    print(page)

Page(text='page_1', number=1)
Page(text='page_2', number=2)
Page(text='page_3', number=3)
Page(text='page_4', number=4)


In [10]:
type(iter(lazy_book))

list_iterator

In [14]:
class PurchasableBook(Book):
    def __init__(self, purchased: bool = False) -> None:
        self.purchased = purchased
        super().__init__()
        
    def __iter__(self) -> "PurchasableBookIter":
        return PurchasableBookIter(self)
        
        
class PurchasableBookIter(BookIter):
    def __init__(self, book):
        self.purchased = book.purchased
        super().__init__(book)
    def __next__(self) -> Page:
        if not self.purchased and self._cursor > 0:
            print("Buy the book to view next pages!")
            raise StopIteration
        return super().__next__()

In [15]:
purchased_book = PurchasableBook()
for i in range(1, 5): 
    purchased_book.add_page(f"page_{i}")       
bookiter = iter(purchased_book)
print(next(bookiter))
print(next(bookiter))

Page(text='page_1', number=1)
Buy the book to view next pages!


StopIteration: 

In [16]:
purchased_book.purchased = True
print(next(bookiter))
print(next(bookiter))
print(next(bookiter))

Buy the book to view next pages!


StopIteration: 

## Is PurchasableBookIter fully match the iterator protocol?

<h2 align=center>Quiz time</h2>

$$
iterators \supset iterable
$$
<center>or</center>
$$
iterators \subset iterable
$$

## Recup

What should an iterator do?

1. Track the current state
1. Know how to return next element
1. ?????
1. <strike>PROFIT</strike> `StopIteration`

Do we realy need to stop?

In [99]:
class RecurrentSequence:
    def __init__(self, a_1: int, a_2: int) -> None:
        self.a_1 = a_1
        self.a_2 = a_2
    
    def __iter__(self) -> Iterator[int]:
        return RecurrentSequenceIterator(self.a_1, self.a_2)

    
class RecurrentSequenceIterator:
    def __init__(self, a_1: int, a_2: int) -> None:
        self.a_1 = a_1
        self.a_2 = a_2
        
    def __iter__(self) -> Iterator[int]:
        return self
    
    def __next__(self) -> int:
        result = self.a_1
        self.a_1, self.a_2 = self.a_2, self.a_1 + self.a_2
        return result

In [278]:
fib = RecurrentSequence(1, 1)
 
for i, f in zip(range(1, 10), fib):
    print(f"{i} - {f}", end="; ")
print()
for i, f in zip(range(1, 10), fib):
    print(f"{i} - {f}", end="; ")

1 - 1; 2 - 1; 3 - 2; 4 - 3; 5 - 5; 6 - 8; 7 - 13; 8 - 21; 9 - 34; 
1 - 1; 2 - 1; 3 - 2; 4 - 3; 5 - 5; 6 - 8; 7 - 13; 8 - 21; 9 - 34; 

In [279]:
fib_iter = iter(fib)

for i, f in zip(range(1, 10), fib_iter):
    print(f"{i} - {f}", end="; ")
print()
for i, f in zip(range(1, 10), fib_iter):
    print(f"{i} - {f}", end="; ")

1 - 1; 2 - 1; 3 - 2; 4 - 3; 5 - 5; 6 - 8; 7 - 13; 8 - 21; 9 - 34; 
1 - 55; 2 - 89; 3 - 144; 4 - 233; 5 - 377; 6 - 610; 7 - 987; 8 - 1597; 9 - 2584; 

## Any side effects?

- one can exhauste an iterator:

In [103]:
iterator = iter([1, 2, 3, 4])
print(sum(iterator))
print(sum(iterator))

10
0


- an iterable has `__contains__` method?

In [171]:
print(list(book))
Page("page_2", 2) in book, Page("page_5", 5) in book

[Page(text='page_1', number=1), Page(text='page_2', number=2), Page(text='page_3', number=3), Page(text='page_4', number=4)]


(True, False)

In [172]:
iterator = iter(book)
Page("page_2", 2) in iterator, Page("page_2", 2) in iterator

(True, False)

In [113]:
5 in fib
# 6 in fib

True

## Any questions so far?

# GENERATORS

Are generators iterators?

In [114]:
def gen():
    yield 1

In [115]:
print(dir(gen()))

['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


Are generators iterators?

\- Yes*

In [119]:
gen, gen(), (1 for _ in [])

(<function __main__.gen()>,
 <generator object gen at 0x000002C1896E9B30>,
 <generator object <genexpr> at 0x000002C1896E9BA0>)

- **Generator functions** - a function or method which uses the yield statement
- **Generator iterator** - an object created by a generator function
- **Generator expression** -an expression that returns an iterator

In [84]:
def recurrent_sequence(a1: int, a2: int):
    while True:
        yield a1
        a1, a2 = a2, a1 + a2
        
fib = recurrent_sequence(0, 1)

for i, f in zip(range(1, 20), fib):
    print(f"{i} - {f}", end="; ")

1 - 0; 2 - 1; 3 - 1; 4 - 2; 5 - 3; 6 - 5; 7 - 8; 8 - 13; 9 - 21; 10 - 34; 11 - 55; 12 - 89; 13 - 144; 14 - 233; 15 - 377; 16 - 610; 17 - 987; 18 - 1597; 19 - 2584; 

## Generators and `return`

In [222]:
def gen():
    yield 1
    return 2

In [223]:
g = gen()
next(g)

1

In [224]:
next(g)

StopIteration: 2

What will happen here?

In [120]:
def gen():
    return 2
    yield 1

In [121]:
gen().__next__()

StopIteration: 2

In [122]:
next(gen(), 'zhaba')

'zhaba'

In [96]:
try:
    next(gen())
except StopIteration as e:
    print(e.value)

2


## Preserving operations order

In [225]:
def do_in_order():
    x = 1
    print(f"Do first, {x}")
    yield
    x += 1
    print(f"Do second, {x}")
    yield
    x *= x
    print(f"Do third, {x}")
gen = do_in_order()
next(gen)
next(gen)

Do first, 1
Do second, 2


In [226]:
next(gen, 'zhaba')

Do third, 4


'zhaba'

## Send

In [17]:
def do_in_order_2():
    x = 1
    print(f"Do first, {x}")
    x = yield
    print(f"Do second, {x}")
    x = yield 42
    print(f"Do third, {x}")

gen = do_in_order_2()
for _ in gen:
    print(f"step {_}")

Do first, 1
step None
Do second, None
step 42
Do third, None


In [105]:
gen = do_in_order_2()
next(gen)
next(gen)
next(gen, "Stop")

Do first, 1
Do second, None
Do third, None


'Stop'

## Send

In [18]:
def do_in_order_2():
    x = 1
    print(f"Do first, {x}")
    x = yield
    print(f"Do second, {x}")
    x = yield 42
    print(f"Do third, {x}")

gen = do_in_order_2()

In [19]:
gen.send(1)

TypeError: can't send non-None value to a just-started generator

In [230]:
gen.send('Hello')

Do second, Hello


42

In [231]:
try:
    gen.send("World")
except StopIteration:
    print("I'm out!")

Do third, World
I'm out!


## Throw

In [237]:
def g():
    try:
        yield 42
    except Exception as e:
        yield e

gen = g()
next(gen)

42

In [238]:
gen.throw(NotImplementedError, "Exception text")

NotImplementedError('Exception text')

In [239]:
gen.throw(NotImplementedError, "Exception text returns")

NotImplementedError: Exception text returns

## Close

In [244]:
def do_in_order_2():
    x = 1
    print(f"Do first, {x}")
    x = yield
    print(f"Do second, {x}")
    x = yield 42
    print(f"Do third, {x}")

gen = do_in_order_2()

In [246]:
next(gen)

Do first, 1


In [247]:
gen.close()

In [248]:
next(gen)

StopIteration: 

## yield from
- pass the execution to another generator
- pass send and throw

In [155]:
def chain(*iterables):
    for iterable in iterables:
        for item in iterable:
            yield item

In [161]:
list(chain([1, 2], ("4", "5"), {"key1": "val1", "key2": "val2"}, "iter"))

[1, 2, '4', '5', 'key1', 'key2', 'i', 't', 'e', 'r']

Can we do better?

In [162]:
def chain(*iterables):
    for iterable in iterables:
        yield from iterable

In [163]:
list(chain([1, 2], ("4", "5"), {"key1": "val1", "key2": "val2"}, "iter"))

[1, 2, '4', '5', 'key1', 'key2', 'i', 't', 'e', 'r']

## Example: Sieve of Eratosthenes

In mathematics, the sieve of Eratosthenes is an ancient algorithm for finding all prime numbers up to any given limit. 

<img src="https://upload.wikimedia.org/wikipedia/commons/b/b9/Sieve_of_Eratosthenes_animation.gif">

1. Create a list of consecutive integers from 2 through n: (2, 3, 4, ..., n).
1. Initially, let p equal 2, the smallest prime number.
1. Enumerate the multiples of p by counting in increments of p from 2p to n, and mark them in the list (these will be 2p, 3p, 4p, ...; the p itself should not be marked).
1. Find the smallest number in the list greater than p that is not marked. If there was no such number, stop. Otherwise, let p now equal this new number (which is the next prime), and repeat from step 3.
1. When the algorithm terminates, the numbers remaining not marked in the list are all the primes below n.

Step 1: generate all natural numbers:

In [164]:
def natural_numbers(start=1):
    while True:
        yield start
        start += 1
        
for _, number in zip(range(10), natural_numbers(1)):
    print(number, end=" ")

1 2 3 4 5 6 7 8 9 10 

Step 2: ~draw the rest of the owl~

In [170]:
def sieve(numbers):
    prime = next(numbers)
    yield prime
    yield from sieve(p for p in numbers if p % prime != 0)
    
for _, prime in zip(range(10), sieve(natural_numbers(2))):
    print(prime, end=" ")

2 3 5 7 11 13 17 19 23 29 

# Lazy evaluations

In [558]:
n = 10**7

In [561]:
%%memit 
sum([i for i in range(n)])

peak memory: 475.96 MiB, increment: 382.55 MiB


In [562]:
%%memit 
sum(i for i in range(n))

peak memory: 94.05 MiB, increment: 0.00 MiB


In [563]:
%%memit
sum(list(map(lambda x: x**2, [i for i in range(n)])))

peak memory: 862.08 MiB, increment: 768.04 MiB


In [564]:
%%memit
sum(map(lambda x: x**2, [i for i in range(n)]))

peak memory: 476.36 MiB, increment: 382.31 MiB


In [565]:
%%memit
sum(map(lambda x: x**2, (i for i in range(n))))

peak memory: 94.14 MiB, increment: 0.00 MiB


# Fin.